# 3. Saving data from the smartphone as CSV

In this tutorial, data uploaded to the server from the iOS application **intdash Motion** is downloaded with the **intdash SDK for Python** (hereinafter referred to as intdash SDK) and saved in CSV format locally. In this case, we will focus on the following.

- Register signal definitions(\*) for general sensor data that convert binary data into numerical values.
- Retrieve data using registered signal definitions
- Convert the time series data to `pandas.DataFrame` and save it as a CSV file

```
Signal definition defines the process of cutting out a part of binary data and reading it as value data such as numerical value and character string.
This process is done on the server side. See the intdash.Conversion-derived class documentation for conversion types
(https://docs.intdash.jp/sdk/python/latest/en/signals.html#intdash.Conversion)
```


## 3.0 Preparation
Before starting this scenario, prepare the following.

- An edge that performs measurement
- A measurement uploaded with intdash Motion (which contains smartphone sensor data)
- Signal definitions for smartphone sensor data


### Data to be used
In this scenario, the following data need to be prepared on the server side.

| Data item | Data name that appears in this scenario |
|:---|:---|
| Edge to register time series data | edge1|
| Signal definitions(\*) | `sp_ACCX`, `sp_ACCY`, `sp_ACCZ`, `sp_Yaw`, `sp_Pitch`, `sp_Roll`|

(\*) For registration method, see the following steps.

### Create measurement data and upload time series data
Upload the data using **intdash Motion**. After registering a measurement, use **Visual M2M Data Visualizer** to check that the new measurement has been created.

For uploading method, refer to **intdash tutorial 2A: Visualizing data from smartphone in real time**.

Confirm that the new measurement is displayed in [Stored Data] of Visual M2M Data Visualizer.
Follow the procedure below to retrieve the time series data of this measurement and save it in CSV format.

<img src="https://github.com/aptpod/aws-marketplace-tutorials/blob/master/img/img4.png?raw=true\">

### Import packages and create a client
For `url` given to `intdash.Client`, specify the environment of the intdash server. For `username` and `password`, specify the auth information issued for the edge you use.

In [20]:
import pandas as pd

import intdash
from intdash import timeutils

# Create client
client = intdash.Client(
    url = "https://example.intdash.jp",
    username = "edge1",
    password="password_here"
)

### Confirm that the signal definitions are registered
Confirm the registered signal definitions.  
If it is not registered, see **(Option) Register the signal definitions** in the next step.

In [7]:
signals = client.signals.list(label='sp')

In [8]:
for s in signals:
    print(s.label,  end=', ')

sp_ACCX, sp_ACCY, sp_ACCZ, sp_Pitch, sp_Roll, sp_Yaw, 

### (Option) Register the signal definitions
```
Warning:
  If the target signal definition has already been registered to the server, skip this step.
```
In this scenario, "general sensor type" data is used. This data is in intdash original format. You need to convert the data from smartphone sensor type to numeric type (Float type this time) on the server side.
In order to perform this conversion, it is necessary to register "signal definitions".
Execute the following file to register the signal definitions.

[0_create-signal-general-sensor.ipynb](../0_create-signal-general-sensor/0_create-signal-general-sensor.ipynb) 

In this sample, register only the conversion definition for 6-axis sensor data.

The preparations are complete.

## 3.1 Retrieve the edge used for measurement
Retrieve the edge used when logging in with intdash Motion.

In [21]:
edges = client.edges.list(name='edge1')
edge1 = edges[0]

In [22]:
edge1.name

'edge1'

## 3.2 Retrieve time series data
Use `client.data_points` to retrieve the time series data.  
For ``labels``, specify the label names of the signal definitions.  
``start`` and ``end`` should be changed to a time that includes the measurements created in the previous step.

In [11]:
dps = client.data_points.list(
    edge_name='edge1',
    start=timeutils.str2timestamp('2020-07-17 00:00:00+09:00'), # change appropriately.
    end=timeutils.str2timestamp('2020-07-18 00:00:00+09:00'), # change appropriately.
    labels=['sp_ACCX', 'sp_ACCY', 'sp_ACCZ', 'sp_Yaw', 'sp_Pitch', 'sp_Roll'],
    limit=0
)

## 3.3 Convert list of DataPoints to a DataFrame

You can convert the data to a DataFrame which has columns for each `data_id` as follows.

In [12]:
from intdash import data

df = pd.DataFrame( [ {
     'time' : d.time,
       d.data_id : data.Float.from_payload(d.data_payload).value # convert binary to numbers.
    }
    for d in dps
]).groupby("time").last()

df

,sp_ACCX,sp_ACCY,sp_ACCZ,sp_Yaw,sp_Pitch,sp_Roll
time,,,,,,
2020-07-17T01:21:45.911747000Z,0.038937,0.491985,0.205379,93.873304,78.406296,3.800612
2020-07-17T01:21:45.931913000Z,0.139753,0.429696,-0.064305,93.758048,78.363416,3.929786
2020-07-17T01:21:45.952079000Z,0.102823,0.398769,-0.076985,93.788928,78.339776,3.883596
2020-07-17T01:21:45.972245000Z,0.046802,0.320168,-0.009198,93.819816,78.322112,3.858561
2020-07-17T01:21:45.992411000Z,0.009810,0.294638,-0.049083,93.846640,78.298808,3.835275
...,...,...,...,...,...,...
2020-07-17T01:22:37.276434000Z,0.295249,0.019525,0.202644,95.308896,20.844832,-8.388880
2020-07-17T01:22:37.296602000Z,0.090947,0.020039,-0.114039,95.321776,20.800294,-8.462379
2020-07-17T01:22:37.316769000Z,-0.232320,-0.185733,-0.398277,95.396584,20.714854,-8.827780


## 3.5 Save in CSV format
Convert to a CSV file using the feature of DataFrame.

In [13]:
df.to_csv('./sample.csv')

The saved file will look like as follows.

<img src="https://github.com/aptpod/aws-marketplace-tutorials/blob/master/img/img1.png?raw=true\">